# Financial Segmentation
---

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('dark_background')

In [2]:
transactions = pd.read_csv('data/twm_transactions.csv', delimiter=';')
accounts = pd.read_csv('data/twm_accounts.csv', delimiter=';')
customers = pd.read_csv('data/twm_customer.csv', delimiter=';')

In [3]:
regions = ['W', 'MW', 'S', 'NE']

state_to_region = {
    'NV': 'W',
    'CA': 'W',
    'NY': 'NE',
    'IL': 'MW',
    'HI': 'W',
    'DC': 'S',
    'TX': 'S',
    'PA': 'NE',
    'OR': 'W',
    'OH': 'MW',
    'WA': 'W',
    'NJ': 'NE',
    'IN': 'MW',
    'CO': 'W',
    'WI': 'MW',
    'MI': 'MW',
    'NB': 'MW',
    'AZ': 'W',
    'MN': 'MW',
    'TN': 'S',
    'VA': 'S',
    'FL': 'S',
    'MD': 'S',
    'MO': 'W',
    'LA': 'S',
    'KY': 'S',
    'OK': 'S',
    'MA': 'NE',
    'GA': 'S',
    'KS': 'MW',
    'NC': 'S',
    'NM': 'W',
    'AL': 'S'
}

## EDA

### Overview

In [4]:
transactions.head()

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
0,27,13625623,0.00,0.00,0.0,3753.34,21.10.1995,121656,A,IQ
1,97,13628392,0.00,0.00,0.0,254.49,5.2.1995,153053,V,IQ
2,21,13630842,-97.57,-97.57,0.0,3819.56,23.7.1995,0,P,WD
3,44,13631412,-0.15,-0.15,0.0,224.05,30.1.1995,0,,FK
4,31,13625722,0.00,0.00,0.0,240.55,25.1.1995,204521,B,IQ


In [5]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77580 entries, 0 to 77579
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tran_id        77580 non-null  int64  
 1   acct_nbr       77580 non-null  int64  
 2   tran_amt       77580 non-null  float64
 3   principal_amt  77580 non-null  float64
 4   interest_amt   77580 non-null  float64
 5   new_balance    77580 non-null  float64
 6   tran_date      77580 non-null  object 
 7   tran_time      77580 non-null  int64  
 8   channel        77580 non-null  object 
 9   tran_code      77580 non-null  object 
dtypes: float64(4), int64(3), object(3)
memory usage: 5.9+ MB


In [6]:
accounts.head()

,acct_nbr,cust_id,acct_type,account_active,acct_start_date,acct_end_date,starting_balance,ending_balance
0,13628063,1362806,SV,Y,10.12.1995,NaN,1430.22,284.58
1,4561143213627090,1362709,CC,Y,15.3.1993,NaN,266.34,496.15
2,4561143213628360,1362836,CC,Y,18.3.1992,NaN,55.90,1000.00
3,13633112,1363311,CK,Y,6.7.1995,NaN,11017.13,968.46
4,4561143213633610,1363361,CC,Y,17.6.1994,NaN,849.37,462.28


In [7]:
accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1409 entries, 0 to 1408
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   acct_nbr          1409 non-null   int64  
 1   cust_id           1409 non-null   int64  
 2   acct_type         1409 non-null   object 
 3   account_active    1409 non-null   object 
 4   acct_start_date   1409 non-null   object 
 5   acct_end_date     143 non-null    object 
 6   starting_balance  1409 non-null   float64
 7   ending_balance    1409 non-null   float64
dtypes: float64(2), int64(2), object(4)
memory usage: 88.2+ KB


In [8]:
print(f'A total of {accounts.cust_id.nunique()} customers hold {accounts.acct_nbr.nunique()} accounts.\nOn average, each customer has {accounts.acct_nbr.nunique()//accounts.cust_id.nunique()} accounts.')

A total of 665 customers hold 1409 accounts.
On average, each customer has 2 accounts.


### Table joining

In [9]:
data = transactions.merge(accounts, 'outer', on='acct_nbr')

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77592 entries, 0 to 77591
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tran_id           77580 non-null  float64
 1   acct_nbr          77592 non-null  int64  
 2   tran_amt          77580 non-null  float64
 3   principal_amt     77580 non-null  float64
 4   interest_amt      77580 non-null  float64
 5   new_balance       77580 non-null  float64
 6   tran_date         77580 non-null  object 
 7   tran_time         77580 non-null  float64
 8   channel           77580 non-null  object 
 9   tran_code         77580 non-null  object 
 10  cust_id           77572 non-null  float64
 11  acct_type         77572 non-null  object 
 12  account_active    77572 non-null  object 
 13  acct_start_date   77572 non-null  object 
 14  acct_end_date     6856 non-null   object 
 15  starting_balance  77572 non-null  float64
 16  ending_balance    77572 non-null  float6

In [11]:
data = data.merge(customers, 'outer', on='cust_id')

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77674 entries, 0 to 77673
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tran_id           77580 non-null  float64
 1   acct_nbr          77592 non-null  float64
 2   tran_amt          77580 non-null  float64
 3   principal_amt     77580 non-null  float64
 4   interest_amt      77580 non-null  float64
 5   new_balance       77580 non-null  float64
 6   tran_date         77580 non-null  object 
 7   tran_time         77580 non-null  float64
 8   channel           77580 non-null  object 
 9   tran_code         77580 non-null  object 
 10  cust_id           77654 non-null  float64
 11  acct_type         77572 non-null  object 
 12  account_active    77572 non-null  object 
 13  acct_start_date   77572 non-null  object 
 14  acct_end_date     6856 non-null   object 
 15  starting_balance  77572 non-null  float64
 16  ending_balance    77572 non-null  float6

### Null filtering

In [13]:
(100 * data.isnull().sum() / len(data)).sort_values(ascending=False)

acct_end_date       91.173366
starting_balance     0.131318
ending_balance       0.131318
acct_start_date      0.131318
account_active       0.131318
acct_type            0.131318
channel              0.121019
tran_code            0.121019
tran_id              0.121019
tran_time            0.121019
tran_date            0.121019
new_balance          0.121019
interest_amt         0.121019
principal_amt        0.121019
tran_amt             0.121019
acct_nbr             0.105569
name_prefix          0.025749
city_name            0.025749
postal_code          0.025749
street_name          0.025749
street_nbr           0.025749
last_name            0.025749
first_name           0.025749
nbr_children         0.025749
marital_status       0.025749
gender               0.025749
cust_id              0.025749
years_with_bank      0.025749
age                  0.025749
income               0.025749
state_code           0.025749
dtype: float64

In [17]:
data[data['acct_end_date'].notnull()]['account_active'].unique()

array(['N '], dtype=object)

All closed accounts are accounted for in the account_active column and acct_end_date is mostly empty. For this analysis, it is safe to discard the acct_end_date feature.

In [18]:
data = data.drop('acct_end_date', axis=1)

In [32]:
data.isnull().any(axis = 1).sum() / data.shape[0] * 100 # Check how many rows have null values relative to all observations.

0.14676725802713908

Since only 114 rows have null values (< 0.15% of the dataframe), it is justifiable to drop the null containing rows.

In [35]:
print(data.shape[0])
data = data.dropna()
print(data.shape[0])

77674
77560


In [40]:
(100 * data.isnull().sum() / len(data)).sort_values(ascending=False) # sanity check

tran_id             0.0
acct_nbr            0.0
city_name           0.0
postal_code         0.0
street_name         0.0
street_nbr          0.0
last_name           0.0
first_name          0.0
name_prefix         0.0
marital_status      0.0
gender              0.0
nbr_children        0.0
years_with_bank     0.0
age                 0.0
income              0.0
ending_balance      0.0
starting_balance    0.0
acct_start_date     0.0
account_active      0.0
acct_type           0.0
cust_id             0.0
tran_code           0.0
channel             0.0
tran_time           0.0
tran_date           0.0
new_balance         0.0
interest_amt        0.0
principal_amt       0.0
tran_amt            0.0
state_code          0.0
dtype: float64

### Categorical feature processing

In [41]:
cat_data = data.select_dtypes('object')

In [42]:
cat_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77560 entries, 0 to 77590
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tran_date        77560 non-null  object
 1   channel          77560 non-null  object
 2   tran_code        77560 non-null  object
 3   acct_type        77560 non-null  object
 4   account_active   77560 non-null  object
 5   acct_start_date  77560 non-null  object
 6   gender           77560 non-null  object
 7   name_prefix      77560 non-null  object
 8   first_name       77560 non-null  object
 9   last_name        77560 non-null  object
 10  street_name      77560 non-null  object
 11  city_name        77560 non-null  object
 12  state_code       77560 non-null  object
dtypes: object(13)
memory usage: 8.3+ MB
